In [6]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file computes LyC escape fractions of the MagE spectra 
# according to Rivera-Thorsen et al. (2019) (arXiv:1904.08186)

In [7]:
import os

from datetime import datetime, timezone

import sigfig

import numpy as np

from scipy.interpolate import interp1d

from astropy.io import fits
from astropy.convolution import convolve_fft, Gaussian2DKernel

from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground, SExtractorBackground, ModeEstimatorBackground, MeanBackground, MMMBackground, BiweightLocationBackground

from reproject import reproject_interp

import stsynphot as STS

import matplotlib.pyplot as plt

In [8]:
def measure():

    '''
    Measure the LyC escape fractions of the MagE apertures
    '''

    def convolve_uncertainties(uncertainty, stdv, shape):

        '''
        Convolve the uncertainties of an image

        Parameters:
            uncertainty : numpy.float64
                The estimated uncertainty of the pixels in the image
            stdv : numpy.float64
                The standard deviation of the Gaussian convolution kernel in pixels
            shape : tuple
                The dimensions of the image

        Returns:
            uncertainty_convolved : numpy.ndarray
                The convolved estimated uncertainty of the pixels in the image, 
                broadcasted to the shape of the image
        '''
    
        # Set the kernel size to 3 standard deviations of the 
        # width of the time-averaged seeing conditions
        kernel_size = int(3 * stdv)

        # Make a dummy array of the convolved uncertainties with the same shape as the image
        uncertainty_convolved = np.ones(shape)

        # Set the squared sum of the weighted uncertainties to zero, to be added to in the double loop below
        squared_sum = 0.0

        # The double loop below calculates the convolved uncertainty of the pixels in the image. 
        # It relies on some assumptions. First, since the original uncertainty is assumed true 
        # for the entire image, the loop does not calculate the convolved uncertainty at each 
        # pixel in the image. It just calculates one instance (since they will all be the same, 
        # ignoring edge effects), and then broadcasts that value to the entire image

        # For each pixel within the kernel size
        for i in range(-kernel_size, kernel_size + 1):
            for j in range(-kernel_size, kernel_size + 1):

                # Calculate the weight of the pixel
                weight = np.exp(-(i**2 + j**2) / (2 * stdv**2))
                weight /= 2 * np.pi * stdv**2

                # Add the weighted uncertainty to the total squared sum of uncertainties
                squared_sum += (weight * uncertainty)**2

        # Multiply the dummy array of the convolved uncertainties by the convolved uncertainty
        uncertainty_convolved = uncertainty_convolved * np.sqrt(squared_sum)

        return uncertainty_convolved

    # Dictionary of the slit IDs, containing, from left to right: the 
    # file name of the data, redshift, and FWHM of the time-averaged 
    # seeing conditions of the observation
    slits = {
        'M5' : ['sunburst_M-5-comb1_MWdr.txt', 2.37086, 0.97],
        'M4' : ['sunburst_M-4-comb1_MWdr.txt', 2.37073, 0.71],
        'M6' : ['sunburst_M-6-comb1_MWdr.txt', 2.37021, 0.76],
        'M3' : ['sunburst_M-3-comb1_MWdr.txt', 2.37025, 0.70],
        'M0' : ['sunburst_M-0-comb1_MWdr.txt', 2.37014, 1.34],
        'M2' : ['sunburst_M-2-comb1_MWdr.txt', 2.37017, 0.77],
        'M7' : ['sunburst_M-7-comb1_MWdr.txt', 2.37044, 0.73],
        'M8' : ['sunburst_M-8-comb1_MWdr.txt', 2.37024, 0.7],
        'M9' : ['sunburst_M-9-comb1_MWdr.txt', 2.37030, 0.68]
    }

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'

    # The IGM transmission according to Rivera-Thorsen et al. (2019) (Science, 366, 738)
    t_IGM = 0.69

    # File paths to masks of the arc and MagE apertures
    arc_mask_file = f'{results}/arc_mask_v5.fits'

    # The v5 reductions of the HST imaging in the F275W and F814W filters, with
    # an intervening galaxy removed by Alex Navarre with GALFIT, and cut to a 
    # smaller footprint containing just the two largest arcs
    f275w_file = f'{data}/hst/galfit_cutouts/f275w_result_residual_v5_wcs.fits'
    f814w_file = f'{data}/hst/galfit_cutouts/f814w_result_residual_v5_wcs.fits'

    # Retrieve the data from the .fits files of the filters 
    f275w = fits.getdata(f275w_file)
    f814w = fits.getdata(f814w_file)

    # The GALFIT outputs are in units of total counts, so divide the data by the
    # total exposure time listed in the .fits header (which is the quantity
    # Alex used to rescale the data into units of total counts). For the F275W
    # imaging, this does not seem to be the same quantity as the total 
    # exposure time that Mike listed in the HST observation log of the paper.
    # I don't know if this was a clerical error made by Mike when tabulating the
    # exposure times actually used, or if the exposure time in the header is
    # the incorrect source. Either way, I don't think it should make a difference
    # when computing the LyC escape fraction as long as I use the scaling factor
    # that Alex did.
    f275w = f275w / 87733
    f814w = f814w / 5280

    # Set the sigma clipping properties
    sigma_clip = SigmaClip(sigma=3.0, maxiters=10)

    # Set the background estimation method
    bkg_estimator = MedianBackground()

    # Create masks of sources in the images
    f275w_source_mask = (f275w >= np.abs(np.median(f275w)))
    f814w_source_mask = (f814w >= 8 * np.abs(np.median(f814w)))
    
    # Create background models of the two images
    f275w_background = Background2D(f275w, (15, 15), filter_size=(3,3), sigma_clip=sigma_clip, 
        bkg_estimator=bkg_estimator, mask=f275w_source_mask, exclude_percentile=80.0, fill_value=0.0)
    f814w_background = Background2D(f814w, (20, 20), filter_size=(3,3), sigma_clip=sigma_clip, 
        bkg_estimator=bkg_estimator, mask=f814w_source_mask, exclude_percentile=20.0, fill_value=0.0)

    # Create a figure that will show the background-subtracted HST images
    fig, ax = plt.subplots(1,2, sharex=True, sharey=True, figsize=(20,10))

    # Plot the background-subtracted HST images, using the source mask to remove sources
    ax[0].imshow(np.ma.masked_where(f275w_source_mask, f275w - f275w_background.background), origin='lower', cmap='viridis', vmin=-0.000001, vmax=0.000001)
    ax[1].imshow(np.ma.masked_where(f814w_source_mask, f814w - f814w_background.background), origin='lower', cmap='viridis', vmin=-0.001, vmax=0.001)

    # Reassign NaNs in the data as zeros
    f275w = np.where(np.isnan(f275w), 0.0, f275w)
    f814w = np.where(np.isnan(f814w), 0.0, f814w)

    # Convert background-subtracted HST images from counts/s to erg/s/cm^2/Å
    f275w = (f275w - f275w_background.background) * fits.open(f'{data}/hst/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits')[0].header['PHOTFLAM']
    f814w = (f814w - f814w_background.background) * fits.open(f'{data}/hst/V5.0_PSZ1G311.65-18.48_F814W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits')[0].header['PHOTFLAM']

    # Retrieve transmission curve data for the HST/WFC3 F275W and HST/ACS F814W filters
    bp_f275w = STS.band('wfc3,uvis1,f275w,mjd#58216')
    bp_f814w = STS.band('acs,wfc1,f814w,mjd#58170')
    w_f275w = bp_f275w.binset
    w_f814w = bp_f814w.binset
    tpt_f275w = bp_f275w(w_f275w)
    tpt_f814w = bp_f814w(w_f814w)
    w_f275w = w_f275w.value
    w_f814w = w_f814w.value

    # Integrate the throughput curves of the HST filters
    int_tpt_f275w = np.trapz(tpt_f275w, x=w_f275w)
    int_tpt_f814w = np.trapz(tpt_f814w, x=w_f814w)

    # Reproject the mask of the arc to the footprint of the image cutouts, since the original arc mask
    # has a much larger footprint
    arc_mask, _ = reproject_interp(fits.open(arc_mask_file)[0], fits.getheader(f275w_file), hdu_in=0)

    # Convert the arc mask to binary values
    arc_mask = np.where(arc_mask > 0, 1, 0)

    # Make empty lists to append the measurement results to
    f275w_fluxes, n_f275w_fluxes, f814w_fluxes, n_f814w_fluxes, f_escs, n_escs = [], [], [], [], [], []

    # For each slit ID
    for i, slit_id in enumerate(slits):
    
        # Get the file name of the MagE data, redshift, and FWHM of the seeing conditions
        file, z, fwhm = slits[slit_id][0], slits[slit_id][1], slits[slit_id][2]
        
        # Reproject the slit mask to the cutout of the HST image, which is much smaller than the footprint of the image with the slit mask
        slit_mask, _ = reproject_interp(fits.open(f'{results}/masks/{slit_id}_mask_v5.fits')[0], fits.getheader(f275w_file))
        
        # Convert the slit mask to binary
        slit_mask = np.where(slit_mask > 0, 1, 0)
    
        # Compute the intersection of the arc and slit masks; preparing both masks in binary format
        # preserves the binary structure in the intersecting mask
        intr = arc_mask * slit_mask

        # Estimate the uncertainties in the two images as the standard deviation of the pixels inside
        # the slit ID's aperture mask but outside the mask of the arc
        n_f275w = np.nanstd(np.where((slit_mask > 0) & (arc_mask < 1), f275w, np.nan))
        n_f814w = np.nanstd(np.where((slit_mask > 0) & (arc_mask < 1), f814w, np.nan))

        # Convert the (assumed) Gaussian FWHM of the time-averaged seeing conditions, expressed 
        # in pixels (in this drizzling, 1 pixel is 0.03 arcseconds), to a standard deviation
        stdv = (fwhm / (2 * np.sqrt(2 * np.log(2)))) / 0.03
    
        # Create a symmetric Gaussian kernel from the standard deviation of the seeing conditions
        kernel = Gaussian2DKernel(x_stddev=stdv, y_stddev=stdv)

        # Convolve the F275W and F814W images with the Gaussian kernel using the fast Fourier 
        # transform method. This saves significant time because of the size of the images
        f275w_convolved = convolve_fft(f275w, kernel)
        f814w_convolved = convolve_fft(f814w, kernel)

        # Convolve the uncertainties, assuming the uncertainty of each pixel 
        # is independent and normally distributed
        n_f275w = convolve_uncertainties(n_f275w, stdv, np.shape(f275w))
        n_f814w = convolve_uncertainties(n_f814w, stdv, np.shape(f814w))

        # Calculate the total flux of the masked slit aperture in each filter
        f275w_flux = np.sum(intr * f275w_convolved * int_tpt_f275w)
        f814w_flux = np.sum(intr * f814w_convolved * int_tpt_f814w)

        # Add the measured fluxes to the corresponding result lists
        f275w_fluxes.append(f275w_flux)
        f814w_fluxes.append(f814w_flux)

        # Calculate the propagated uncertainty of the flux measurements
        n_f275w_flux = (abs(f275w_flux) / np.sum(intr * f275w_convolved)) * np.sqrt(np.sum((abs(intr * f275w_convolved) * n_f275w / f275w_convolved)**2))
        n_f814w_flux = (abs(f814w_flux) / np.sum(intr * f814w_convolved)) * np.sqrt(np.sum((abs(intr * f814w_convolved) * n_f814w / f814w_convolved)**2))

        # Add the measured flux uncertainties to the corresponding result lists
        n_f275w_fluxes.append(n_f275w_flux)
        n_f814w_fluxes.append(n_f814w_flux)

        # Get the Starburst99 fit's wavelength bins and flux densities
        w_sb99, f_sb99 = np.loadtxt(f'{data}/mage/stellar_continuum_fits/planckarc-{slit_id}-sb99-fit.txt', comments='#', usecols=(0,3), unpack=True)

        # Get the wavelength bins and flux densities and associated uncertainties of the MagE spectrum
        w, f, n = np.loadtxt(f'{data}/mage/{file}', delimiter='\t', comments=('#','w'), usecols=(0,1,2), unpack=True)
    
        # Create a mask excluding extreme flux density outliers
        f_mask = f < 1e-20    

        # Apply the mask to the data to remove data points with extreme
        # flux density outliers
        w, f, n = w[f_mask], f[f_mask], n[f_mask]

        # Convert the flux densities and associated uncertainties to 
        # wavelength space (from erg/s/cm^2/Hz to erg/s/cm^2/Å)
        f, n = f * 2.998e18 / np.square(w), n * 2.998e18 / np.square(w)
    
        # Convert the MagE spectrum data to the rest frame
        w, f, n = w / (1 + z), f * (1 + z), n * (1 + z)
    
        # Scale the Starburst99 fit to match the MagE spectrum (the Starburst99 fits were 
        # normalized by the median flux density between 1267-1276 Å in the rest frame)
        f_sb99 = f_sb99 * np.median(f[(w >= 1267) & (w <= 1276)])

        # Calculate the uncertainty of the Starburst99 flux densities, 
        # conservatively assuming a constant 10% uncertainty
        n_sb99 = 0.1 * f_sb99
    
        # Convert the Starburst99 fit to the observed frame
        w_sb99, f_sb99, n_sb99 = w_sb99 * (1 + z), f_sb99 / (1 + z), n_sb99 / (1 + z)

        # Create masks of the wavelength bounds of the two HST filters
        w_sb99_f275w_mask = (w_sb99 >= 2100) & (w_sb99 <= 3250)
        w_sb99_f814w_mask = (w_sb99 >= 6500) & (w_sb99 <= 10000)

        # Mask the Starburst99 fits to the wavelength bounds of the two HST filters
        w_sb99_f275w, f_sb99_f275w, n_sb99_f275w = w_sb99[w_sb99_f275w_mask], f_sb99[w_sb99_f275w_mask], n_sb99[w_sb99_f275w_mask]
        w_sb99_f814w, f_sb99_f814w, n_sb99_f814w = w_sb99[w_sb99_f814w_mask], f_sb99[w_sb99_f814w_mask], n_sb99[w_sb99_f814w_mask]
    
        # Interpolate the throughput curves of the HST filters to the Starburst99 observed-frame wavelength bins
        interpT275 = interp1d(w_f275w, tpt_f275w)
        tpt275interp = interpT275(w_sb99_f275w)
        interpT814 = interp1d(w_f814w, tpt_f814w)
        tpt814interp = interpT814(w_sb99_f814w)
    
        # Integrate the Starburst99 flux densities as transmitted through the filters
        f_sb99_f275w_trnsmtd = np.trapz(tpt275interp * f_sb99_f275w, x=w_sb99_f275w)
        f_sb99_f814w_trnsmtd = np.trapz(tpt814interp * f_sb99_f814w, x=w_sb99_f814w)
    
        # Make an empty list that will contain the uncertainty of each trapezoidal area of the 
        # integration of the transmitted, F275W-masked Starburst99 fit
        n_areas = []
    
        # For each wavelength bin in the Starburst99 fit masked by the F275W filter wavelengths
        for j, wav in enumerate(w_sb99_f275w[:-1]):
        
            # Calculate the width of the wavelength bin
            w_bin_width = w_sb99_f275w[j+1] - w_sb99_f275w[j]
        
            # Following the trapezoidal interpretation of integration, determine the area and 
            # associated uncertainty of the trapezoid starting at the jth wavelength bin 
            n_area = abs((w_bin_width * (tpt275interp[j] * f_sb99_f275w[j] + tpt275interp[j+1] * f_sb99_f275w[j+1]) / 2.0)) / (tpt275interp[j] * f_sb99_f275w[j] + tpt275interp[j+1] * f_sb99_f275w[j+1]) \
                * np.sqrt((abs(tpt275interp[j] * f_sb99_f275w[j]) * n_sb99_f275w[j] / f_sb99_f275w[j])**2 + (abs(tpt275interp[j+1] * f_sb99_f275w[j+1]) * n_sb99_f275w[j+1] / f_sb99_f275w[j+1])**2)
        
            # Add the measured uncertainty to the uncertainty list
            n_areas.append(n_area)
        
        # Calculate the uncertainty of the integration of the transmitted, F275W-masked Starburst99 fit
        n_int3 = np.sqrt(np.sum(np.array(n_areas, dtype=np.float64) ** 2))

        # Make an empty list that will contain the uncertainty of each trapezoidal area of the 
        # integration of the transmitted, F814W-masked Starburst99 fit
        n_areas = []
    
        # For each wavelength bin in the Starburst99 fit masked by the F814W filter wavelengths
        for j, wav in enumerate(w_sb99_f814w[:-1]):
        
            # Calculate the width of the wavelength bin
            w_bin_width = w_sb99_f814w[j+1] - w_sb99_f814w[j]

            # Following the trapezoidal interpretation of integration, determine the area and 
            # associated uncertainty of the trapezoid starting at the jth wavelength bin 
            n_area = abs((w_bin_width * (tpt814interp[j] * f_sb99_f814w[j] + tpt814interp[j+1] * f_sb99_f814w[j+1]) / 2.0)) / (tpt814interp[j] * f_sb99_f814w[j] + tpt814interp[j+1] * f_sb99_f814w[j+1]) \
                * np.sqrt((abs(tpt814interp[j] * f_sb99_f814w[j]) * n_sb99_f814w[j] / f_sb99_f814w[j])**2 + (abs(tpt814interp[j+1] * f_sb99_f814w[j+1]) * n_sb99_f814w[j+1] / f_sb99_f814w[j+1])**2)

            # Add the measured uncertainty to the uncertainty list
            n_areas.append(n_area)
        
        # Calculate the uncertainty of the integration of the transmitted, F814W-masked Starburst99 fit
        n_int4 = np.sqrt(np.sum(np.array(n_areas, dtype=np.float64) ** 2))
    
        # Calculate the LyC escape fraction and append it to the measurement results list
        f_esc = (f275w_flux * int_tpt_f275w * f_sb99_f814w_trnsmtd) / (t_IGM * f814w_flux * int_tpt_f814w * f_sb99_f275w_trnsmtd)
        f_escs.append(f_esc)
    
        # Calculate the propagated uncertainty of the LyC escape fraction and append it to the uncertainty list
        n_esc = abs(f_esc) * np.sqrt((n_f275w_flux / f275w_flux)**2 + (n_f814w_flux / f814w_flux)**2 + (n_int3 / f_sb99_f275w_trnsmtd)**2 + (n_int4 / f_sb99_f814w_trnsmtd)**2)
        n_escs.append(n_esc)
    
    # Create a header for the LyC escape fraction measurements output file
    header = 'Measurements of the LyC escape fraction of each MagE slit aperture\n' \
        + f'Created by esc.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}\n' \
        + '\n' \
        + 'Columns, from left to right: HST/WFC3 F275W flux and uncertainty (erg/s/cm^2), HST/ACS F814W flux and uncertainty (erg/s/cm^2), and LyC escape fraction and uncertainty (%)'

    # Save the results to an output file
    np.savetxt(f'{results}/f_esc_lyc_measurements.txt', np.array([f275w_fluxes, n_f275w_fluxes, f814w_fluxes, n_f814w_fluxes, f_escs, n_escs], dtype=np.float64).T, delimiter=' ', header=header, comments='# ')

def tabulate(): 

    '''
    Make a table of the LyC escape fraction measurements
    '''    

    def round_to_uncertainties(measurement, uncertainty):

        '''
        Round the uncertainty to one significant figure and the measurement to
        the same significant digit as the uncertainty

        Parameters:

            measurement : numpy.float64
                Value of the measurement

            uncertainty : numpy.float64
                Estimated uncertainty of the measurement

        Returns:

            measurement : str
                Value of the measurement, rounded to the same significant digit as
                the 1 significant figure-rounded uncertainty

            uncertainty : str
                Estimated uncertainty of the measurement, rounded to 1 significant figure
        '''

        # Round the uncertainty to one significant figure
        uncertainty = sigfig.round(uncertainty, sigfigs=1, type=str)

        # If the uncertainty is less than 1
        if '.' in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, decimals=len(uncertainty.split('.')[1]), type=str)

        # If the uncertainty is greater than 1
        elif '.' not in uncertainty:

            # Round the measurement to the same digit as the uncertainty
            measurement = sigfig.round(measurement, len(str(measurement).replace('-','').split('.')[0]) - len(uncertainty) + 1, type=str)

        return measurement, uncertainty

    # Dictionary of the slit IDs
    slits = {
        'M5' : [],
        'M4' : [],
        'M6' : [],
        'M3' : [],
        'M0' : [],
        'M2' : [],
        'M7' : [],
        'M8' : [],
        'M9' : []
    }

    # Establish common directories
    home = os.getcwd()
    results = f'{home}/results'

    # Create the table header
    table = '\\begin{deluxetable}{cccc}[t]\n' \
        + '\n' \
        + '\\tablecaption{HST-based properties of the MagE spectra \label{tab:f_esc}}\n' \
        + '\n' \
        + '\\tablehead{\n' \
        + '\t\colhead{Slit} & \colhead{$F_{275}$} & \colhead{$F_{814}$} & \colhead{$f_{\\rm{esc}}^{\\rm{LyC}}$}\n' \
        + '}\n' \
        + '\n' \
        + '\startdata\n\n'

    # Open the LyC escape fraction measurements of the slits
    f_esc_lyc_measurements = np.loadtxt(f'{results}/f_esc_lyc_measurements.txt', delimiter=' ', comments='#')

    for i in range(4):
        f_esc_lyc_measurements[:,i] = f_esc_lyc_measurements[:,i] * 1e19

    # For each slit ID
    for i, slit_id in enumerate(slits):

        # Get the LyC escape fraction measurements for the slit ID
        slit_f_esc_lyc_measurements = f_esc_lyc_measurements[i]

        # Add the slit ID to the row
        table = table + f'{slit_id} '

        # For each measurement type to list in the table
        for j in range(3):

            # Get the measurement and its uncertainty for the slit ID
            measurement = slit_f_esc_lyc_measurements[2 * j]
            uncertainty = slit_f_esc_lyc_measurements[2 * j + 1]

            # Round the uncertainty to 1 significant figure and the measurement to the 
            # corresponding significant digit
            measurement, uncertainty = round_to_uncertainties(measurement, uncertainty)

            # Add the formatted measurement / uncertainty to the table row
            table = table + f'& ${measurement}\pm{uncertainty}$ '

        # Add a LaTeX row break character, unless the row is the last row in the table
        table = table + '\\\\\n' if i != len(slits) - 1 else table + '\n'

    # Add a footer to the table
    table = table + '\n\enddata\n' \
        + '\n' \
        + '\\tablecomments{From left to right: slit label, flux in the HST/WFC3 F275W and HST/ACS F814W filters ($10^{-20}$ erg s$^{-1}$ cm$^{-2}$), and $f_{\\text{esc}}^{\\text{LyC}}$ (\%), all computed according to \S\,\\ref{ssec:methods_fesc}.}\n' \
        + '\n' \
        + '\end{deluxetable}'

    # Save the table
    f = open(f'{results}/tables/f_esc_lyc_measurements_table.txt', 'w', encoding='utf-8')
    f.write(table)
    f.close()

In [ ]:
measure()

In [10]:
tabulate()